<h1>Import Libraries

In [15]:
import sys
from paddleocr import PaddleOCR
import cv2
import numpy as np  
import pandas as pd
import os
from doctr.models import ocr_predictor
from torch.utils.data import DataLoader
from doctr.io import DocumentFile


print(sys.version)
ocr = PaddleOCR(lang='en')
model = ocr_predictor(pretrained=True)



3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:52:09) 
[Clang 14.0.6 ]
[2024/06/25 20:57:18] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/s03.rathorgmail.com/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/s03.rathorgmail.com/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_p

<h1>Image Processing and Predicting Marks

<h3>Helper functions

In [16]:
def find_surr_keys(unassigned_key, known_keys):
    # Sort known keys
    print(known_keys)
    known_keys = sorted(known_keys)
    # Initialize distances and closest keys
    closest_keys = []
    for k in known_keys:
         closest_keys.append((abs(int(k) - int(unassigned_key)), k))
    # Sort by distance
    closest_keys.sort()
    # Return the two closest known keys
    if(closest_keys[0][1]<unassigned_key and closest_keys[0][1]>unassigned_key):
        return [closest_keys[0][1], closest_keys[1][1]]
    else:
        raise ValueError(f"No closest keys found for unassigned key: {unassigned_key}")

def label_text(text):
    # Define the two lists
    list1 = ['t', 'r', 'u', 'T', 'R', 'U']
    list2 = ['f', 'a', 'l', 's', 'F', 'A', 'L', 'S']
    
    # Count the matches for each list
    count1 = sum(text.count(char) for char in list1)
    count2 = sum(text.count(char) for char in list2)
    
    # Determine the label based on the counts
    if count1 > count2:
        return True
    elif count1!=0 or count2!=0:
        return False


def percentMatch(text1,text2):
    list = ['t', 'r', 'u', 'T', 'R', 'U','f', 'a', 'l', 's', 'F', 'A', 'L', 'S']

    if(text1):
        count1 = sum(text1.count(char) for char in list)
       
        
        count2 = sum(text2.count(char) for char in list)

        if(count1==3 and count2==4 or count1==4 and count2==3 ):  #if one says true and other says false then priority given to 2nd
            print("true and false collision so given priority to text2 which is the incoming text")
            return 2

        if(count1>count2):
            print("text1 i.e the prev text is the winner")
            return 1
        else:
            print("text2 i.e the incoming text is the winner")
            return 2
    else:
        print("text1 not there so text2 is the winner")
        return 2

def count_true_false(d): #in a dictionary to check how many T/F are there.
    true_count = sum(1 for v in d.values() if v is True)
    false_count = sum(1 for v in d.values() if v is False)
    return true_count, false_count

def merge_dicts(dict1, dict2): 
    true_count1, false_count1 = count_true_false(dict1)
    true_count2, false_count2 = count_true_false(dict2)
    
    if (true_count1 + false_count1) >= (true_count2 + false_count2):
        final_dict = dict1.copy()
        y_dirn_gap=False
    else:
        final_dict = dict2.copy()
        y_dirn_gap=True

    return final_dict,y_dirn_gap


In [17]:
def assign_true_false_or_unknown(true_list, false_list, question_dict):
    # Initialize the final dictionary
    final_dict = {str(i): 'UNASSIGNED' for i in range(1, 11)}
    unassigned_keys=[]
    assigned_keys=[]
    
    # Iterate over each question and its y-coordinate
    for question, y in question_dict.items():
        # compute diff with true list such that we sub t/f box from s/n box
        true_differences= [y - ty for ty in true_list]
        # Compute absolute differences with true list
        true_abs_differences = [abs(y - ty) for ty in true_list]
        # Compute absolute differences with false list
        false_differences= [y - ty for ty in false_list]
        false_abs_differences = [abs(y - fy) for fy in false_list]


        # Find the minimum differences
  
        min_true_diff = min((diff for diff in true_differences if diff > 0), default=float('inf'))
        min_false_diff = min((diff for diff in false_differences if diff > 0), default=float('inf'))

        
        min_true_abs_diff=min(true_abs_differences) if true_differences else float('inf')
        min_false_abs_diff=min(false_abs_differences) if false_differences else float('inf')

        # Determine the smallest difference
        min_diff = min(min_true_diff, min_false_diff)
        min_abs_diff=min(min_true_abs_diff,min_false_abs_diff)


        # Assign the value based on the smallest difference
        if min_diff < 360:
            if min_true_diff < min_false_diff:
                final_dict[question] = True
                true_list.pop(true_differences.index(min_true_diff))
            else:
                final_dict[question] = False
                false_list.pop(false_differences.index(min_false_diff))
        else:
            # checking the abs diff option if nothing can find in positive diff option
            if min_abs_diff < 360:
                if min_true_abs_diff < min_false_abs_diff:
                    final_dict[question] = True
                    true_list.pop(true_abs_differences.index(min_true_abs_diff))
                else:
                    final_dict[question] = False
                    false_list.pop(false_abs_differences.index(min_false_abs_diff))
            
            else:
                final_dict[question] = 'NULL'
   
    print("the final dict without editing",final_dict)
    
    # solving the case when question number is not detected but answer is detected
    for key, value in final_dict.items():
        if value == "UNASSIGNED":
            unassigned_keys.append(key)
        else:
            assigned_keys.append(key)
    
    closest_keys_dict={}
    print("the unassigned keys are",unassigned_keys)
    print("the assigned keys are",assigned_keys)
    
    
    
    try:
        for key, value in final_dict.items():
            if value == "UNASSIGNED":
                unassigned_keys.append(key)
            else:
                assigned_keys.append(key)
        
        closest_keys_dict={}
        # print("the unassigned keys are",unassigned_keys)
        # print("the assigned keys are",assigned_keys)
        for key in unassigned_keys:
            #    print("the key is: ",key)
            if key=='1':
                question_dict[key]=question_dict['2']-300
            elif key=='10':
                question_dict[key]=question_dict['9']+300
            else:
                lesser_key, greater_key = find_surr_keys(key,assigned_keys)
                closest_keys_dict[key] = (lesser_key, greater_key)

                diff=int(greater_key)-int(lesser_key)
                # print(lesser_key)
                # print(greater_key)
                

                index=int(key)-int(lesser_key)-1
                lowest_adj_index=int((diff-1)/2)
                adj_mult=(-lowest_adj_index)+index
                
                if((diff-1)%2==0):
                    if(adj_mult>=0):
                        adj_mult=adj_mult+1
                
                # print("the number which is used for adjusting ",adj_mult)
                question_dict[key]=((question_dict[lesser_key]+question_dict[greater_key])/2)+(300*adj_mult)

            

        for question, y in question_dict.items():
            if question in unassigned_keys:

                print("that question which is editted is",question)
                print("the value of y is",y)
                # Compute absolute differences with true list
                true_differences = [abs(y - ty) for ty in true_list]
                # Compute absolute differences with false list
                false_differences = [abs(y - fy) for fy in false_list]
                
                # Find the minimum differences
                min_true_diff = min(true_differences) if true_differences else float('inf')
                min_false_diff = min(false_differences) if false_differences else float('inf')
                
                # Determine the smallest difference
                min_diff = min(min_true_diff, min_false_diff)
                
                # Assign the value based on the smallest difference
                if min_diff < 360:
                    if min_true_diff < min_false_diff:
                        final_dict[question] = True
                        true_list.pop(true_differences.index(min_true_diff))
                    else:
                        final_dict[question] = False
                        false_list.pop(false_differences.index(min_false_diff))
                else:
                    final_dict[question] = 'NULL'
    
    
    except Exception as e:
        return final_dict
                           
    
    
    
    
    return final_dict


In [18]:
def assign_true_false_or_unknown_rotated(true_list,false_list,true_list_x,false_list_x,question_dict,question_dict_x):
    final_dict = {str(i): 'UNASSIGNED' for i in range(1, 11)}
    unassigned_keys=[]
    assigned_keys=[]
    final_dict_y={str(i): 'UNASSIGNED' for i in range(1, 11)}
    final_dict_x={str(i): 'UNASSIGNED' for i in range(1, 11)}

    # Iterate over each question and its y-coordinate
    for question, y in question_dict.items():
        # Compute absolute differences with true list
        true_differences= [y - ty for ty in true_list]
        true_abs_differences = [abs(y - ty) for ty in true_list]
        # Compute absolute differences with false list
        false_differences= [y - fy for fy in false_list]
        false_abs_differences = [abs(y - fy) for fy in false_list]


        # Find the minimum differences
  
        min_true_diff = min((diff for diff in true_differences if diff > 0), default=float('inf'))
        min_false_diff = min((diff for diff in false_differences if diff > 0), default=float('inf'))

        
        min_true_abs_diff=min(true_abs_differences) if true_abs_differences else float('inf')
        min_false_abs_diff=min(false_abs_differences) if false_abs_differences else float('inf')

        # Determine the smallest difference
        min_diff = min(min_true_diff, min_false_diff)
        min_abs_diff=min(min_true_abs_diff,min_false_abs_diff)

        # print("the question number is :",question)
        # print("the min dist is :",min_diff)
        # print("the min abs_diff is :",min_abs_diff)
        # print("the false abs diff",false_abs_differences)
        


        # Assign the value based on the smallest difference first going with abs diff as for upside down it will favour abs
        if min_abs_diff < 310:
            if min_true_abs_diff < min_false_abs_diff:
                final_dict_y[question] = True
                true_list.pop(true_abs_differences.index(min_true_abs_diff))
            else:
                final_dict_y[question] = False
                false_list.pop(false_abs_differences.index(min_false_abs_diff))
        else:
            # checking the postive diff option if nothing can find in abs diff option
            if min_diff < 310:
                print(question)
                if min_true_diff < min_false_diff:

                    final_dict_y[question] = True
                    true_list.pop(true_differences.index(min_true_diff))
                else:
                    final_dict_y[question] = False
                    false_list.pop(false_differences.index(min_false_diff))
            
            else:
                final_dict_y[question] = 'NULL'
   



    for question,x in question_dict_x.items():
    
        # Compute absolute differences with true list
        true_differences= [x - tx for tx in true_list_x]
        true_abs_differences = [abs(x - tx) for tx in true_list_x]
        # Compute absolute differences with false list
        false_differences= [x - fy for fy in false_list_x]
        false_abs_differences = [abs(x - fy) for fy in false_list_x]


        # Find the minimum differences
  
        min_true_diff = min((diff for diff in true_differences if diff > 0), default=float('inf'))
        min_false_diff = min((diff for diff in false_differences if diff > 0), default=float('inf'))

        
        min_true_abs_diff=min(true_abs_differences) if true_abs_differences else float('inf')
        min_false_abs_diff=min(false_abs_differences) if false_abs_differences else float('inf')

        # Determine the smallest difference
        min_diff = min(min_true_diff, min_false_diff)
        min_abs_diff=min(min_true_abs_diff,min_false_abs_diff)

        if min_diff < 310:
            if min_true_diff < min_false_diff:
                final_dict_x[question] = True
                true_list_x.pop(true_differences.index(min_true_diff))
            else:
                final_dict_x[question] = False
                false_list_x.pop(false_differences.index(min_false_diff))
        else:
            # checking the abs diff option if nothing can find in positive diff option
            if min_abs_diff < 310:
                if min_true_abs_diff < min_false_abs_diff:
                    final_dict_x[question] = True
                    true_list_x.pop(true_abs_differences.index(min_true_abs_diff))
                else:
                    final_dict_x[question] = False
                    false_list_x.pop(false_abs_differences.index(min_false_abs_diff))
            
            else:
                final_dict_x[question] = 'NULL'
    
    
    print("the final dict for y is: ")
    print(final_dict_y)
    print("the final dict for x is: ")
    print(final_dict_x)
    final_dict,y_dirn_gap=merge_dicts(final_dict_x,final_dict_y)

   
    if 'L' in final_dict:
        final_dict['7']=final_dict['L']
        del final_dict['L']
        
    if 'I' in final_dict:
        final_dict['1']=final_dict['I']
        del final_dict['I']
    
    if y_dirn_gap and '6' in final_dict and '9' in final_dict: #means image is inverted and 6 and 9 true and false value needs to swapped out
        temp=final_dict['6']
        final_dict['6']=final_dict['9']
        final_dict['9']=temp





        
    print("the final dict without editing",final_dict) 

    try:
        for key, value in final_dict.items():
            if value == "UNASSIGNED":
                unassigned_keys.append(key)
            else:
                assigned_keys.append(key)
        
        closest_keys_dict={}
        # print("the unassigned keys are",unassigned_keys)
        # print("the assigned keys are",assigned_keys)
        for key in unassigned_keys:
            #    print("the key is: ",key)
            if y_dirn_gap:
                if key=='1':
                    question_dict[key]=question_dict['2']-300
                elif key=='10':
                    question_dict[key]=question_dict['9']+300
                else:
                    lesser_key, greater_key = find_surr_keys(key,assigned_keys)
                    closest_keys_dict[key] = (lesser_key, greater_key)

                    diff=int(greater_key)-int(lesser_key)
                    # print(lesser_key)
                    # print(greater_key)
                

                    index=int(key)-int(lesser_key)-1
                    lowest_adj_index=int((diff-1)/2)
                    adj_mult=(-lowest_adj_index)+index
                
                    if((diff-1)%2==0):
                        if(adj_mult>=0):
                            adj_mult=adj_mult+1
                
                    # print("the number which is used for adjusting ",adj_mult)
                    question_dict[key]=((question_dict[lesser_key]+question_dict[greater_key])/2)+(300*adj_mult)

            else:
                if key=='1':
                    question_dict_x[key]=question_dict_x['2']-300
                elif key=='10':
                    question_dict_x[key]=question_dict_x['9']+300
                else:
                    lesser_key, greater_key = find_surr_keys(key,assigned_keys)
                    closest_keys_dict[key] = (lesser_key, greater_key)

                    diff=int(greater_key)-int(lesser_key)
                    print(lesser_key)
                    print(greater_key)
                

                    index=int(key)-int(lesser_key)-1 #index  of missing number
                    lowest_adj_index=int((diff-1)/2)
                    adj_mult=(-lowest_adj_index)+index
                
                    if((diff-1)%2==0):
                        if(adj_mult>=0):
                            adj_mult=adj_mult+1
                
                    # print("the number which is used for adjusting ",adj_mult)
                    question_dict_x[key]=((question_dict_x[lesser_key]+question_dict_x[greater_key])/2)+(300*adj_mult)

            
        if y_dirn_gap:
            for question, y in question_dict.items():
                if question in unassigned_keys:

                    print("that question which is editted is",question)
                    print("the value of y is",y)
                    # Compute absolute differences with true list
                    true_differences = [abs(y - ty) for ty in true_list]
                    # Compute absolute differences with false list
                    false_differences = [abs(y - fy) for fy in false_list]
                    
                    # Find the minimum differences
                    min_true_diff = min(true_differences) if true_differences else float('inf')
                    min_false_diff = min(false_differences) if false_differences else float('inf')
                    
                    # Determine the smallest difference
                    min_diff = min(min_true_diff, min_false_diff)
                    
                    # Assign the value based on the smallest difference
                    if min_diff < 350:
                        if min_true_diff < min_false_diff:
                            final_dict[question] = True
                            true_list.pop(true_differences.index(min_true_diff))
                        else:
                            final_dict[question] = False
                            false_list.pop(false_differences.index(min_false_diff))
                    else:
                        final_dict[question] = 'NULL'
        else:
            for question,x in question_dict_x.items():
                if question in unassigned_keys:
                    print("that question which is editted is",question)
                    print("the value of x is",x)
                    # Compute absolute differences with true list
                    true_differences = [abs(y - ty) for ty in true_list_x]
                    # Compute absolute differences with false list
                    false_differences = [abs(y - fy) for fy in false_list_x]
                    
                    # Find the minimum differences
                    min_true_diff = min(true_differences) if true_differences else float('inf')
                    min_false_diff = min(false_differences) if false_differences else float('inf')
                    
                    # Determine the smallest difference
                    min_diff = min(min_true_diff, min_false_diff)
                    
                    # Assign the value based on the smallest difference
                    if min_diff < 350:
                        if min_true_diff < min_false_diff:
                            final_dict[question] = True
                            true_list_x.pop(true_differences.index(min_true_diff))
                        else:
                            final_dict[question] = False
                            false_list_x.pop(false_differences.index(min_false_diff))
                    else:
                        final_dict[question] = 'NULL'
                    
    
    
    
    except Exception as e:
        return final_dict
                           
    
    
    
    
    return final_dict


<h2>Using PaddleOCR

In [22]:
def process_using_paddleocr(image_path,output_folder,output_folder1):
    
    ocr = PaddleOCR(lang='en')
    base_name = os.path.basename(image_path)
    image_cv = cv2.imread(image_path)
    print("!------------------------------start with paddleocr-----------------------------------!")
    print("Started processing of the image :",base_name)
    
    output = ocr.ocr(image_path)[0]


    
    texts = [line[1][0] for line in output]
   
    #checking for rotation
    r_count=0

    while('TRUE/FALSE' not in texts):
        image_cv = cv2.rotate(image_cv, cv2.ROTATE_90_CLOCKWISE)

        # Save the rotated image to a temporary path
        temp_image_path = 'temp_rotated_image.jpg'
        cv2.imwrite(temp_image_path, image_cv)
        output=ocr.ocr(temp_image_path)[0]
        texts = [line[1][0] for line in output]
        print(texts)
        r_count+=1
        if r_count==4:  #reaching the same orientation
            break
    
    if(r_count>0 and r_count!=4):
        cv2.imwrite(image_path,image_cv)
        print("rotation done for: ",base_name)
        print("Number of times rotation done:",r_count)

    
    print("OCR detection done")

    boxes = [line[0] for line in output]
    # probabilities = [line[1][1] for line in output]



    image_boxes = image_cv.copy()



    # print("!------------------------------all coordinates-----------------------------------!")

    for box,text in zip(boxes,texts):

      cv2.rectangle(image_boxes,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  #needs top left and bottom right to draw bounding box
      cv2.putText(image_boxes,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),3)
    
    




    output_file_name = f'detect_{base_name}'
    output_file_path = os.path.join(output_folder1, output_file_name)
    
    # Save the processed image
    cv2.imwrite(output_file_path, image_boxes)
  


    for box, text in zip(boxes, texts):
      if text=="SN" or text=="NS":
        num_l_x1=box[0][0]
        num_r_x1=box[2][0]+140
        num_l_y1=box[0][1]
        num_r_y1=box[2][1]+140
        print("left top x of SN:",num_l_x1)
        print("bottom right x of SN:",num_r_x1)
        print("left top y of SN:",num_l_y1)
        print("bottom right y of SN:",num_r_y1)

      if text=="TRUE/FALSE":
        num_l_x2=box[0][0]-10
        num_r_x2=box[2][0]+20
        num_l_y2=box[0][1]-140
        num_r_y2=box[2][1]+140
        print("left top x of T/F:",num_l_x2)
        print("bottom right x of T/F:",num_r_x2)
        print("left top y of T/F:",num_l_y2)
        print("bottom right y of T/F:",num_r_y2)



        cons_boxes_image=image_cv.copy()
        true_list=[]
        false_list=[]

        true_list_x=[]
        false_list_x=[]


        true_false_list=[]
        numbers_dict={}
        numbers_dict_x={}
        c=0
        prev_x=0
        prev_y=0


        # this is for s/n column
        try:
            for box, text in zip(boxes, texts):
                # print(f"the text is : {text}")

                
                box_top_left_x = int(box[0][0])
                box_top_left_y=int(box[0][1])
                box_bottom_right_x = int(box[2][0])
                box_bottom_right_y = int(box[2][1])
                box_width_x = box_bottom_right_x - box_top_left_x
                box_width_y = box_bottom_right_y - box_top_left_y
                if (num_l_x1 <= box_bottom_right_x <= num_r_x1 or num_l_y1<= box_bottom_right_y<=num_r_y1) and box_width_x <= 200 and box_width_y <= 200 and text!="SN" and text!="NS":
                    # print("entered in the S/N column ")
                    # print(text)
                    # print(box)
                    numbers_dict[text] = int(box[0][1])
                    numbers_dict_x[text] = int(box[0][0])

                    cv2.rectangle(cons_boxes_image, (int(box[0][0]), int(box[0][1])), (int(box[2][0]), int(box[2][1])), (0, 0, 255), 5)
                    cv2.putText(cons_boxes_image, text, (int(box[0][0]), int(box[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 4, (222, 0, 0), 1)


                    #removing the text and boxes from the list
                    # texts.remove(text)
                    # boxes.remove(box)



        
        
        
            try:
                for box, text in zip(boxes, texts):

                    box_top_left_x = int(box[0][0])
                
                    box_bottom_right_x = int(box[2][0])
                    box_bottom_right_y = int(box[2][1])
                    box_width=box_bottom_right_x-box_top_left_x
                
                # Draw the adjusted bounding box
                    if (num_l_x2 <=  box_bottom_right_x <= num_r_x2 and box_width<=1200):
                        # print("entered in the t/f column ")
                    
                        if label_text(text)==True and text!='TRUE/FALSE':
                            if(c==0):
                                # print("first t/f detection")
                                # print(text)
                                # print(box)
                                prev=box[0][1]
                                true_list.append(int(box[0][1]))
                            else:
                            
                                if(abs(box[0][1]-prev)>160):  #to avoid boxes in same row to overlap 
                                    # print(text)
                                    # print(box)
                                    true_list.append(int(box[0][1]))
                                    prev=box[0][1]
                            c+=1
                        
                        elif label_text(text)==False and text!='TRUE/FALSE':
                            if(c==0):
                                # print("first t/f detection")
                                # print(text)
                                # print(box)
                                prev=box[0][1]
                                false_list.append(int(box[0][1]))
                            else:
                                if(abs(box[0][1]-prev)>160):
                                    # print(text)
                                    # print(box)
                                    false_list.append(int(box[0][1]))
                                    prev=box[0][1]
                            c+=1

                        cv2.rectangle(cons_boxes_image,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  
                        cv2.putText(cons_boxes_image,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),1)
            
                
    
                final_dict=assign_true_false_or_unknown(true_list,false_list,numbers_dict)
                 # Create a unique output file name
                output_file_name = f'final_tf_{base_name}'
                output_file_path = os.path.join(output_folder, output_file_name)
    
                # Save the processed image
                cv2.imwrite(output_file_path, cons_boxes_image)




            
            except NameError:

                print("TRUE/FALSE not detected. Skipping this part of processing.")
                print("going with all detection of t/f")

                for box, text in zip(boxes, texts):
                    box_top_left_x = int(box[0][0])
                    box_top_left_y = int(box[0][1])
                    box_bottom_right_x = int(box[2][0])
                    box_bottom_right_y = int(box[2][1])

                    box_width=box_bottom_right_x-box_top_left_x
                    box_width_y=box_bottom_right_y-box_top_left_y


                    # print(box)
                    # print(text)


                    if((box_width<=800 and box_width_y<=800) and label_text(text)==True):
                        if(c==0):
                                print("first t/f detection")
                                # print(text)
                                # print(box)
                                prev_y=box[0][1]
                                prev_x=box[0][0]
                                true_list.append(int(box[0][1]))
                                true_list_x.append(int(box[0][0]))
                                
                        else:
                                
                            if((abs(box[0][0]-prev_x)>250) or  abs(box[0][1]-prev_y)>250):
                                # print(text)
                                # print(box)
                                true_list.append(int(box[0][1]))
                                true_list_x.append(int(box[0][0]))
                                prev_y=box[0][1]
                                prev_x=box[0][0]
                                
                            c+=1

                        cv2.rectangle(cons_boxes_image,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  
                        cv2.putText(cons_boxes_image,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),1)
                    
                    if((box_width<=800 and box_width_y<=800) and label_text(text)==False):
                        if(c==0):
                                print("first t/f detection")
                                print(text)
                                print(box)
                                prev_y=box[0][1]
                                prev_x=box[0][0]

                        #         false_list.append(int(box[0][1]))
                        #         false_list_x.append(int(box[0][0]))
                        else:
                                
                            if((abs(box[0][0]-prev_x)>250) or  abs(box[0][1]-prev_y)>250):
                                print(text)
                                print(box)
                                false_list.append(int(box[0][1]))
                                false_list_x.append(int(box[0][0]))
                                prev_y=box[0][1]
                                prev_x=box[0][0]
                            
                            c+=1
                        
                        cv2.rectangle(cons_boxes_image,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  
                        cv2.putText(cons_boxes_image,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),1)
            

                print("the true list is ",true_list)
                print("the false list is ",false_list)
                print("the true list for xdirn",true_list_x)
                print("the false list for xdirn",false_list_x)

                final_dict=assign_true_false_or_unknown_rotated(true_list,false_list,true_list_x,false_list_x,numbers_dict,numbers_dict_x)
                
                 # Create a unique output file name
                output_file_name = f'final_tf_{base_name}'
                output_file_path = os.path.join(output_folder, output_file_name)
    
                # Save the processed image
                cv2.imwrite(output_file_path, cons_boxes_image)


        #error in detection of S/N column   
        except NameError:
            print("cant detect s/n column also so going with all detection using box width")
            c=0
            for box,text in zip(boxes,texts):
                box_top_left_x = int(box[0][0])
                box_top_left_y=int(box[0][1])
                box_bottom_right_x = int(box[2][0])
                box_bottom_right_y = int(box[2][1])
                box_width_x = box_bottom_right_x - box_top_left_x
                box_width_y = box_bottom_right_y - box_top_left_y
                
                
                if (box_width_x <= 200 and box_width_y <= 200):
                    if text.isdigit():
                        number = int(text)
                        if 1 <= number <= 10:
                            # Store in dictionaries only if the number is between 1 and 10
                            numbers_dict[text] = int(box[0][1])
                            numbers_dict_x[text] = int(box[0][0])

                            # Visualize the rectangle and text on the image (optional)
                            cv2.rectangle(cons_boxes_image, (int(box[0][0]), int(box[0][1])), (int(box[1][0]), int(box[1][1])), (0, 0, 255), 5)
                            cv2.putText(cons_boxes_image, text, (int(box[0][0]), int(box[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 4, (222, 0, 0), 1)

                if((box_width_x<=800 and box_width_y<=800) and label_text(text)==True):
                    if(c==0):
                            print("first t/f detection")
                            # print(text)
                            # print(box)
                            prev_y=box[0][1]
                            prev_x=box[0][0]
                            true_list.append(int(box[0][1]))
                            true_list_x.append(int(box[0][0]))
                                
                    else:
                                
                        if((abs(box[0][0]-prev_x)>250) or  abs(box[0][1]-prev_y)>250):
                            # print(text)
                            # print(box)
                            true_list.append(int(box[0][1]))
                            true_list_x.append(int(box[0][0]))
                            prev_y=box[0][1]
                            prev_x=box[0][0]
                            
                        c+=1

                    cv2.rectangle(cons_boxes_image,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  
                    cv2.putText(cons_boxes_image,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),1)
                    
                if((box_width_x<=800 and box_width_y<=800) and label_text(text)==False):
                    if(c==0):
                            print("first t/f detection")
                            print(text)
                            print(box)
                            prev_y=box[0][1]
                            prev_x=box[0][0]

                    #         false_list.append(int(box[0][1]))
                    #         false_list_x.append(int(box[0][0]))
                    else:
                            
                        if((abs(box[0][0]-prev_x)>250) or  abs(box[0][1]-prev_y)>250):
                            print(text)
                            print(box)
                            false_list.append(int(box[0][1]))
                            false_list_x.append(int(box[0][0]))
                            prev_y=box[0][1]
                            prev_x=box[0][0]
                        
                        c+=1
                    
                    cv2.rectangle(cons_boxes_image,(int(box[0][0]),int(box[0][1])),(int(box[2][0]),int(box[2][1])),(0,0,255),5)  
                    cv2.putText(cons_boxes_image,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),1)
            

            print("the true list is ",true_list)
            print("the false list is ",false_list)
            print("the true list for xdirn",true_list_x)
            print("the false list for xdirn",false_list_x)

            final_dict=assign_true_false_or_unknown_rotated(true_list,false_list,true_list_x,false_list_x,numbers_dict,numbers_dict_x)
             # Create a unique output file name
            output_file_name = f'final_tf_{base_name}'
            output_file_path = os.path.join(output_folder, output_file_name)
    
            # Save the processed image
            cv2.imwrite(output_file_path, cons_boxes_image)




    return final_dict

                  

<h2>Using Doctr

In [25]:
def process_and_save_image(image_path,csv_path,pred_csv_path ,output_folder , output_folder1):
    base_name = os.path.basename(image_path)
    image_cv = cv2.imread(image_path)
    height = image_cv.shape[0]
    width = image_cv.shape[1]
    print("!------------------------------starting detection using doctr-----------------------------------!")
    print("Started processing of the image :",base_name)
    # print(image_width)
    # output = ocr.ocr(image_path)[0]

    single_img_doc = DocumentFile.from_images(image_path)
    result = model(single_img_doc)

    

    texts=[]
 
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                for word in line.words:
                    text = word.value
                    texts.append(text)
  


   
    #checking for rotation
    r_count=0

    while('TRUE/FALSE' not in texts):
        image_cv = cv2.rotate(image_cv, cv2.ROTATE_90_CLOCKWISE)
        print("rotation started")


        # Save the rotated image to a temporary path
        # temp_image_path = 'temp_rotated_image.jpg'
        cv2.imwrite(image_path, image_cv)

        # output=ocr.ocr(temp_image_path)[0]
        single_img_doc = DocumentFile.from_images(image_path)
        result=model(single_img_doc)
        texts=[]
        for page in result.pages:
            for block in page.blocks:
                for line in block.lines:
                    for word in line.words:
                        text = word.value
                        texts.append(text)

        print(texts)
        r_count+=1
        if r_count==4:  #reaching the same orientation
            break
    


    if(r_count>0 and r_count!=4):
        # cv2.imwrite(image_path,image_cv)
        print("rotation done for: ",base_name)
        print("Number of times rotation done:",r_count)

    height = image_cv.shape[0]
    width = image_cv.shape[1]
    
    print("OCR detection done with doctr")
    boxes=[]
    # boxes = [line[0] for line in output]4
    for page in result.pages:
        for block in page.blocks:
            for line in block.lines:
                for word in line.words:
                   
                    (x_min, y_min), (x_max, y_max) = word.geometry
                    
                    x_min_px = x_min * width
                    y_min_px = y_min * height
                    x_max_px = x_max * width
                    y_max_px = y_max * height
                    
                    bbox=(x_min_px, y_min_px), (x_max_px, y_max_px)

                  
                    boxes.append(bbox)

    image_boxes = image_cv.copy()



    # print("!------------------------------all coordinates-----------------------------------!")

    for box,text in zip(boxes,texts):
    #   print(text)
    #   print(box)
      cv2.rectangle(image_boxes,(int(box[0][0]),int(box[0][1])),(int(box[1][0]),int(box[1][1])),(0,0,255),5)  #needs top left and bottom right to draw bounding box
      cv2.putText(image_boxes,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,4,(222,0,0),3)
    
    
    # print("!------------------------------done with all coordinates-----------------------------------!")



    output_file_name = f'detect_{base_name}'
    output_file_path = os.path.join(output_folder1, output_file_name)
    
    # Save the processed image
    cv2.imwrite(output_file_path, image_boxes)
  


    for box, text in zip(boxes, texts):
      if text=="SN" or text=="NS":
        num_l_x1=box[0][0]-40
        num_r_x1=box[1][0]+140
    
        print("left top x of SN:",num_l_x1)
        print("bottom right x of SN:",num_r_x1)
       

      if text=="TRUE/FALSE":
        num_l_x2=box[0][0]-10
        num_r_x2=box[1][0]+200
       
      
        print("left top x of T/F:",num_l_x2)
        print("bottom right x of T/F:",num_r_x2)




    


    # Draw OCR bounding boxes within the final rectangle
    cons_boxes_image=image_cv.copy()
    true_list=[]
    false_list=[]



    numbers_dict={}
    numbers_dict_x={}
    c=0
   

    try:

        # this is for s/n column
        for box, text in zip(boxes, texts):
            # print(f"the text is : {text}")

        
            box_top_left_x = int(box[0][0])
            box_top_left_y=int(box[0][1])
            box_bottom_right_x = int(box[1][0])
            box_bottom_right_y = int(box[1][1])
     

        
            # print(box_bottom_right_x)
            # print(box_bottom_right_y)
            # print(box_width_x)
            # print(box_width_y)

 
        
            if (num_l_x1 <=  box_bottom_right_x <= num_r_x1 ):
                if text.isdigit():
                    number = int(text)
                    if 1 <= number <= 10:
                        # Store in dictionaries only if the number is between 1 and 10
                        numbers_dict[text] = int(box[0][1])

                        # Visualize the rectangle and text on the image (optional)
                        cv2.rectangle(cons_boxes_image, (int(box[0][0]), int(box[0][1])), (int(box[1][0]), int(box[1][1])), (0, 0, 255), 5)
                        cv2.putText(cons_boxes_image, text, (int(box[0][0]), int(box[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (222, 0, 0), 1)


  



    
    
     
        
        prev=0


    
    
        for box, text in zip(boxes, texts):

            
        
            box_bottom_right_x = int(box[1][0])
    
        
        # Draw the adjusted bounding box
            if (num_l_x2 <=  box_bottom_right_x <= num_r_x2):
                # print("entered in the t/f column ")
            
                if label_text(text)==True and text!='TRUE/FALSE':
                    if(c==0):
                        print("first t/f detection")
                        print(text)
                        print(box)
                        prev=box[0][1]
                        prev_text=text
                        true_list.append(int(box[0][1]))
                    else:
                    
                        if(abs(box[0][1]-prev)>160):  #to avoid boxes in same row to overlap 
                            print(text)
                            print(box)
                            true_list.append(int(box[0][1]))
                            prev=box[0][1]
                            prev_text=text
                        else:
                            print(f"collision happend with box:{prev} and text:{prev_text} solving on the basis of percent match boxes")
                            print("the current box specification are")
                            print(text)
                            print(box)
                            ans=percentMatch(prev_text,text)
                            if(ans==2):

                                if(label_text(prev_text)==False):
                                    false_list.pop()
                            
                                elif(label_text(prev_text)==True):
                                    true_list.pop()
                            
                            
                                prev=box[0][1]
                                prev_text=text
                                true_list.append(int(prev))
                            
                            
                                

                    c+=1
                
                elif label_text(text)==False and text!='TRUE/FALSE':
                    if(c==0):
                        print("first t/f detection")
                        print(text)
                        print(box)
                        prev=box[0][1]
                        prev_text=text
                        false_list.append(int(box[0][1]))
                    else:
                        if(abs(box[0][1]-prev)>160):
                            print(text)
                            print(box)
                            false_list.append(int(box[0][1]))
                            prev=box[0][1]
                            prev_text=text
                        else:
                            print(f"collision happend with box:{prev} and text:{prev_text} solving on the basis of percent match boxes")
                            print("the current box specification are")
                            print(text)
                            print(box)
                            ans=percentMatch(prev_text,text)

                            if(ans==2):

                                if(label_text(prev_text)==False):
                                    false_list.pop()
                            
                                elif(label_text(prev_text)==True):
                                    true_list.pop()
                            
                            
                                prev=box[0][1]
                                prev_text=text
                                false_list.append(int(prev))
                    c+=1

                cv2.rectangle(cons_boxes_image,(int(box[0][0]),int(box[0][1])),(int(box[1][0]),int(box[1][1])),(0,0,255),5)  
                cv2.putText(cons_boxes_image,text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)
    
            
  
        final_dict=assign_true_false_or_unknown(true_list,false_list,numbers_dict)
    
        # Create a unique output file name
        output_file_name = f'final_tf_{base_name}'
        output_file_path = os.path.join(output_folder, output_file_name)
    
        # Save the processed image
        cv2.imwrite(output_file_path, cons_boxes_image)


        print("printing the number dict y_coordinate")
        print(numbers_dict)
     



    
    except NameError:

        print("TRUE/FALSE not detected. Skipping this part of processing.")
        print("going with all detection")

        final_dict=process_using_paddleocr(image_path,output_folder,output_folder1)

                


    
        

   



    
    print("--------- Printing the final dict ------------")
    print(final_dict)

   
    


        
        
    



    predictions_file_path=pred_csv_path
    # reading the answers and evaluting 
    actualAns_df=pd.read_csv(csv_path)

    marks=0

    for index, row in actualAns_df.iterrows():
        question_number = str(row["Question Number"])
        answer = row["Correct Answer"]
        print(answer)
        if question_number in final_dict and final_dict[question_number]==answer:
            marks += 1

    print("Total Marks:", marks)

    image_name = base_name #Replace this with the actual image name
    marks_df = pd.DataFrame({"Filename": [image_name], "Marks": [marks]})
    

    # Append the marks DataFrame to the predictions file
    marks_df.to_csv(predictions_file_path, mode='a', header=False, index=False)



<h1>Specifying the File Paths

In [27]:
# Define the path to the folder containing images
parent_folder='Phase1Dataset'
# parent_folder='mytest_V2'

# Image folder name
image_folder = f'{parent_folder}/Data_1_335'
# eg: image_folder = f'{parent_folder}/Sample_Data'

#Detection folder where images with  bounding box on all detected words will be stored
detection_folder = os.path.join(parent_folder, 'detection')

#Output directory where processed images with bounding boxes for question numbers and True/False answers will be saved
output_folder = os.path.join(parent_folder, 'output')


# Create the new folders if they don't already exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(detection_folder, exist_ok=True)


# path of csv where the output will be written
pred_csv_path=f'{parent_folder}/csvs/submission1.csv'


#Answer Key for the question paper
# actual_ans_csv_path=f'{parent_folder}/ModelAnswer.csv'

with open(pred_csv_path,'w') as fd:
    fd.write('img_name,pred_marks\n')
    fd.close()


#mapping csv path
mapping_csv_path=f'{parent_folder}/img_model_answer_mapping.csv'
mapping_df = pd.read_csv(mapping_csv_path)

marks_model_csv_1 = f'{parent_folder}/model_answer_type1.csv'
marks_model_csv_2 = f'{parent_folder}/model_answer_type2.csv'


for file_name in os.listdir(image_folder):
    # Construct the full file path
    file_path = os.path.join(image_folder, file_name)
    
    # Check if the file is an image (you can add more extensions if needed)
    if file_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        
        csv_choice = mapping_df.loc[mapping_df['img_name'] == file_name, 'model_answer'].values

        if csv_choice == "model_answer_type1":
            actual_ans_csv_path = marks_model_csv_1
        elif csv_choice == "model_answer_type2":
            actual_ans_csv_path = marks_model_csv_2


        try:
            process_and_save_image(file_path, actual_ans_csv_path,pred_csv_path,output_folder,detection_folder)
        
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue


print("Processing complete. Images saved to:", output_folder,detection_folder)

!------------------------------starting detection using doctr-----------------------------------!
Started processing of the image : 2EufWcq.jpg
rotation started
['E', '8', 'N', 'D3', 'E', 'I', '2', 'F', 'F', '5', 'I', 'P', 'I', 'E', 'I', 'I', 'p.', '8', 'I', 'e', '7', 'H', 'E', 'E', 'I', 'I', 'I', 'E', ':', 'I', 'i', '[', '8', 'I', '2', 'I', 'E', 'e', 'E', 'I', '8', 'a', 'I', '€', 'I', 'e', 'I', '!', '2', 'I', 'g', '1', 'I', '&', 'LE', 'I', 'I', 'E', 'E', 'F', 'I', '9', 'a', 'I', 'I', 'a', 'E', 'B', 'I', 'I', '8', 'a', 'f', '5', 'I', 'I', 'I', 'I', '8', '2', 'E', '1', 'I', 'E', 'i', 'I', 'E', '1', '&', 'E', 'I', 'E', 'E', 'I', 'g', '8', 'it', 'I', 'I', 'I', 'I', 'I', ':', 'I', 'of', 'LE', '8', 'I', '1', 'E', 'I', '£', 'le', '3', '-', 'I', 'I', 'I', 'I', 'I', '2', 'E', 'I', 'e', '1E', 'P', 'A', 'e', 'I', 'E', 'I', 'E', 'e', 'E', 'I', 'I', 'I', 'iE', 'I', 'E', 'I', 'F', 'F', 'I', '1', 'I', 'I', '7', 'E', 'o', '2', '', 'E', 'E', 'I', ':', '1', '2', 'I', 'I', 'I', 'I', 'I', 'E', '2', '1E',

<h1>Evaluation(MAE)

In [11]:
import pandas as pd



pred_df=pd.read_csv(pred_csv_path)

#specify the path where actual marks to image files are given
actual_df=pd.read_csv(f'{parent_folder}/Marks.csv')



error=0
count=0

for index, row in pred_df.iterrows():
    filename=row["img_name"]
    actual_row=actual_df.loc[actual_df['Filename']==filename]
    actual_marks=actual_row['Total Marks'].values[0]
    pred_marks=row['pred_marks']
    diff=abs(pred_marks-actual_marks)
    if(diff>0):
        print(f"Filename: {actual_row['Filename'].values[0]}")
        print(f"Diff: {actual_marks-pred_marks}")
        count+=1
    error+=diff
    

print(len(pred_df))
print(len(actual_df))
print(f"no of files in which error: {count}")

print("the mean absolute error is: ",error/len(pred_df))

Filename: 20240328_150223.jpg
Diff: 1
Filename: 20240328_153456.jpg
Diff: 1
Filename: 20240328_145247.jpg
Diff: -1
Filename: 20240328_153338.jpg
Diff: 2
Filename: 20240328_155246.jpg
Diff: 1
47
48
no of files in which error: 5
the mean absolute error is:  0.1276595744680851
